<div>
    <a href="https://colab.research.google.com/github/ellacharmed/machine-learning-zoomcamp/blob/week08/cohorts/2023/08-deep-learning/homework-08.ipynb">
        <img src="https://colab.research.google.com/assets/colab-badge.svg" width="120" height="25">
    </a>
    <a href="https://app.community.saturnenterprise.io/dash/o/community/resources?templateId=8e5976027b6a4f5e8f98c40574ded639">
        <img src="https://saturncloud.io/images/embed/run-in-saturn-cloud.svg" width="120" height="25">
    </a>
</div>

## Homework 10-kubernetes

In this homework, we'll deploy the credit scoring model from the homework 5.
We already have a docker image for this model - we'll use it for 
deploying the model to Kubernetes.


## Bulding the image

Clone the course repo if you haven't:

```
git clone https://github.com/DataTalksClub/machine-learning-zoomcamp.git
```

Go to the `course-zoomcamp/cohorts/2023/05-deployment/homework` folder and 
execute the following:


```bash
docker build -t zoomcamp-model:hw10 .
```

> **Note:** If you have troubles building the image, you can 
> use the image we built and published to docker hub:
> `docker pull svizor/zoomcamp-model:hw10`

### Question 1 : `0.7269`

Run it to test that it's working locally:

```bash
docker run -it --rm -p 9696:9696 zoomcamp-model:hw10
```


In [11]:
%cd "/Users/ellaair/MEGA/Projects/machine-learning-zoomcamp/cohorts/2023/05-deployment/homework"

/Users/ellaair/MEGA/Projects/machine-learning-zoomcamp/cohorts/2023/05-deployment/homework


In [12]:
%pwd

'/Users/ellaair/MEGA/Projects/machine-learning-zoomcamp/cohorts/2023/05-deployment/homework'

And in another terminal, execute `q6_test.py` file:

```bash
python q6_test.py
```

You should see this:

```python
{'get_credit': True, 'get_credit_probability': <value>}
```

Here `<value>` is the probability of getting a credit card. You need to choose the right one.

* 0.3269
* 0.5269
* 0.7269
* 0.9269

Now you can stop the container running in Docker.

In [13]:
!python qn6_test.py

{'approval_probability': 0.726936946355423, 'loan approved': True}


## Installing `kubectl` and `kind`

You need to install:

* `kubectl` - https://kubernetes.io/docs/tasks/tools/ (you might already have it - check before installing)
* `kind` - https://kind.sigs.k8s.io/docs/user/quick-start/


### Question 2 : `0.20.0`

What's the version of `kind` that you have? 

Use `kind --version` to find out.


In [14]:
!kind --version

kind version 0.20.0


## Creating a cluster

Now let's create a cluster with `kind`:

```bash
kind create cluster
```

And check with `kubectl` that it was successfully created:

```bash
kubectl cluster-info
```


In [15]:
!kubectl get service

NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   22s


### Question 3 : `10.96.0.1`

Now let's test if everything works. Use `kubectl` to get the list of running services. 

What's `CLUSTER-IP` of the service that is already running there? 


In [16]:
!kind load docker-image zoomcamp-model:hw10

Image: "zoomcamp-model:hw10" with ID "sha256:23acf085d7ff4faa067679a1cc4b5abf145d3fb9b4da8b9d2646cb74640360b0" not yet present on node "kind-control-plane", loading...


### Question 4 : `kind load docker-image`

To be able to use the docker image we previously created (`zoomcamp-model:hw10`),
we need to register it with `kind`.

What's the command we need to run for that?

* `kind create cluster`
* `kind build node-image`
* `kind load docker-image`
* `kubectl apply`



### Question 5 : `9696`

Now let's create a deployment config (e.g. `deployment.yaml`):

```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: credit
spec:
  selector:
    matchLabels:
      app: credit
  replicas: 1
  template:
    metadata:
      labels:
        app: credit
    spec:
      containers:
      - name: credit
        image: <Image>
        resources:
          requests:
            memory: "64Mi"
            cpu: "100m"            
          limits:
            memory: <Memory>
            cpu: <CPU>
        ports:
        - containerPort: <Port>
```

Replace `<Image>`, `<Memory>`, `<CPU>`, `<Port>` with the correct values.

What is the value for `<Port>`?

Apply this deployment using the appropriate command and get a list of running Pods. 
You can see one running Pod.

```bash
kubectl apply -f deployment.yaml
kubectl get pods

> bank-667887d95c-jdhmr
```

### Question 6 : `<???> = bank`

Let's create a service for this deployment (`service.yaml`):

```yaml
apiVersion: v1
kind: Service
metadata:
  name: <Service name>
spec:
  type: LoadBalancer
  selector:
    app: <???>
  ports:
  - port: 80
    targetPort: <PORT>
```

Fill it in. What do we need to write instead of `<???>`?

Apply this config file.

```bash
kubectl apply -f service.yaml
```

## Testing the service

We can test our service locally by forwarding the port 9696 on our computer 
to the port 80 on the service:

```bash
kubectl port-forward service/<Service name> 9696:80
```

Run `q6_test.py` (from the homework 5) once again to verify that everything is working. 
You should get the same result as in Question 1.

```bash
kubectl port-forward service/credit-service 9696:80
```

In [17]:
!python qn6_test.py

{'approval_probability': 0.726936946355423, 'loan approved': True}
